In [1]:
import scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import pickle
import math

In [2]:
fDF = pd.read_csv('Data/featureTypes.txt', names=['featureID'])

In [3]:
print fDF.shape
print fDF['featureID'][0]
n = 10000
d = 3000

(3000, 1)
flavors raspberries cherries


In [4]:
trainDF = pd.read_csv('Data/trainData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
YDF = pd.read_csv('Data/trainLabels.txt', names = ['label'])
valXDF = pd.read_csv('Data/valData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
valYDF = pd.read_csv('Data/valLabels.txt', names = ['label'])
print trainDF.shape[0]

247847


In [5]:
W = np.random.rand(d)  #random.uniform(low=0.0, high=1.0,size = (d,))
B = 0 #np.zeros(n)
TempBArray = np.ones(n)
print W.shape
#print TempBArray
print np.count_nonzero(W)
print len(W)
print W

(3000,)
3000
3000
[ 0.67658663  0.90802376  0.48529491 ...,  0.97657844  0.60761507
  0.74764667]


In [6]:
trainDF[:5]

,instanceID,featureID,value
0,1,13,0.209
1,1,83,0.209
2,1,228,0.209
3,1,242,0.209
4,1,371,0.209


In [7]:
#tDF = csr_matrix(trainDF) 
#print tDF[:5]

In [8]:
#sdf = pd.SparseDataFrame(tDF)
#print sdf[:5]

In [9]:
# Will lead to negative index if re-running
trainDF['instanceID'] -= 1
trainDF['featureID'] -= 1
sMat = csr_matrix((trainDF['value'], (trainDF['featureID'], trainDF['instanceID'])))
valXDF['instanceID'] -= 1
valXDF['featureID'] -= 1
valX = csr_matrix((valXDF['value'], (valXDF['featureID'], valXDF['instanceID'])))
Y = YDF['label'].as_matrix().transpose()
#print Y.shape
valY = valYDF['label'].as_matrix().transpose()

In [10]:
#print sMat.shape
#print sMat.todense()
print sMat.max(), sMat.min()

0.728 0.0


In [11]:
#print sMat[:2]

In [12]:
X = sMat.copy()
print X.shape

(3000, 10000)


In [13]:
print len(W.nonzero())
print len(W)

1
3000


In [14]:
def initLamda(X, Y):
    YNorm = Y - float(Y.sum())/((float)(Y.shape[0]))
    return 2 * (abs(X * YNorm).max())
    
print initLamda(X, Y)

1271.869651


In [15]:
t = X.copy()
t.data **= 2
At = 2*t.sum(axis = 1)
print At

[[ 0.731852]
 [ 2.660682]
 [ 1.581472]
 ..., 
 [ 1.564942]
 [ 2.31796 ]
 [ 1.5809  ]]


In [16]:
redFact = 0.000002

In [17]:
# change this to convergence condition

def rmse(input1, input2):
    out = input1 - input2
    #print out
    out **= 2
    out /= len(out)
    error = out.sum()
    return math.sqrt(error)


class Lasso:
    def __init__(self, X, Y, W, B, Lamda):
        self.X = X.copy()
        self.Y = Y.copy()
        self.W = W #.copy()  # Remove this copy later
        self.B = B #.copy()
        t = X.copy()
        t.data **= 2
        self.A = 2*t.sum(axis = 1)
        self.Lamda = Lamda
        self.delta = 0.01
        # Stores Lamda and respective RMSE
        self.trainrmse = []
        self.trainlamda = []
        self.valrmse = []
        self.vallamda = []
        self.NonZero = []
        
    def loss(self):
        return ((self.X.transpose() * self.W + self.B - self.Y) ** 2).sum() + self.Lamda * (abs(self.W)).sum()
        
    def fit(self):
        # Lamda = initLamda(self.X, self.Y)
        
        #print X.shape, W.shape
        #for epoch in range(100):
        oldLoss = self.loss()+2
        newLoss = self.loss()
        print 'Lamda: ', self.Lamda
        while oldLoss - newLoss > self.delta:
        #for i in range(1000):
            #print sMat.transpose() * W
            # 4.1.1
            #print t1[:5], t1.shape
            #print t1.shape, B.shape, Y.shape
            XTW = (self.X.transpose() * self.W)
            R = self.Y - (self.X.transpose() * self.W) - self.B
            
            # 4.1.2
            BOld = self.B
            self.B = np.full(n, (R + self.B).sum() / n) 
            #print self.B
            #print self.B.shape
            #self.B = (self.Y - XTW).sum() / n
            #print B.shape
            # 4.1.3
            R =  R + BOld - self.B
            #print 'RSHAPE', R.shape
            #print 'R', R
            #R = self.Y - (XTW + self.B)
            #print R.shape
            #print R[:5]
            # R = R.reshape(-1)
            for ik in range(0, d):
                # 4.1.4
                #ik = 0
                t = (self.X[ik].transpose() * self.W[ik]).toarray().reshape(-1)
                #print t
                #print t.shape
                #print R.shape
                Ck = 2*( self.X[ik] * (R + t)).sum()
                #print 'CK:', Ck
                # Update Weight
                WkOld = self.W[ik]
                #print 'OW: ', WkOld
                if Ck < -self.Lamda:
                    self.W[ik] = (Ck + self.Lamda) / self.A[ik]
                elif Ck > self.Lamda:
                    self.W[ik] = (Ck - self.Lamda) / self.A[ik]
                else:
                    self.W[ik] = 0
                #print 'W: ', WkOld, self.W[ik]
                #print W[ik]
                # 4.1.5
                # print self.W[ik], WkOld
                #print X[ik].toarray().reshape(-1).shape, R.shape
                R = R + self.X[ik].toarray().reshape(-1) * (WkOld - self.W[ik])
                #R = self.Y - (self.X.transpose() * self.W) + self.B
            oldLoss = newLoss
            newLoss = model.loss()
            #print oldLoss, newLoss, oldLoss - newLoss
            print 'LOSS:' , newLoss
            # End of feature vector iterator
    
    def saveModel(self, filename):
        pickle.dump(self, open( filename, "wb" ))
    
    def predict(self, X):
        return (X.transpose() * self.W + np.full(X.transpose().shape[0], self.B))
    
    def chooseCorrectLamda(self, delta = -1):
        oldLamda = self.Lamda
        if delta != -1:
            self.delta = delta
        self.fit()
        
        newRMSE = rmse(self.predict(self.X), self.Y)
        #self.TrainInfo.append([self.Lamda, newRMSE])
        self.trainrmse.append(newRMSE)
        self.trainlamda.append(self.Lamda)
        valRMSE = rmse(self.predict(valX), valY)
        self.valrmse.append(valRMSE)
        self.vallamda.append(self.Lamda)
        oldRMSE = valRMSE
        #print W
        #self.ValInfo.append([self.Lamda, valRMSE])
        self.NonZero.append((self.W != 0.0).sum())
        print 'Lamda: ', self.Lamda, 'RMSE: ', newRMSE, 'Val RMSE:' , valRMSE
        
        while oldRMSE >= valRMSE:
            oldLamda = self.Lamda
            if self.Lamda < 6:
                self.Lamda -= redFact
            else:
                self.Lamda /= 2;
            redFact
            self.fit()
            oldRMSE = valRMSE
            #self.TrainInfo.append([self.Lamda, newRMSE])
            newRMSE = rmse(self.predict(self.X), self.Y)
            self.trainrmse.append(newRMSE)
            self.trainlamda.append(self.Lamda)
            valRMSE = rmse(self.predict(valX), valY)
            #self.ValInfo.append([self.Lamda, valRMSE])
            self.valrmse.append(valRMSE)
            self.vallamda.append(self.Lamda)
            self.NonZero.append(np.count_nonzero(self.W))
            #self.NonZero.append(self.W.toarray().count_nonzero())
            print 'Lamda: ', self.Lamda, 'RMSE: ', newRMSE, 'Val RMSE:' , valRMSE
            self.saveModel('optimal_saved_Model')
    
def loadModel(filename):
    return pickle.load(open(filename, "rb" ))

In [18]:
model = Lasso(X, Y, W, B, 2.858711) #initLamda(X.copy(), Y.copy()))
#model.loss()
#model.fit()

In [19]:
print model.Lamda

2.858711


In [20]:
#model.Lamda = 2.85899999
model.chooseCorrectLamda()

Lamda:  2.858711
LOSS: 53025.0660606
LOSS: 38546.6460746
LOSS: 36151.1892829
LOSS: 35486.1891463
LOSS: 35241.4378372
LOSS: 35133.1865411
LOSS: 35075.7222237
LOSS: 35039.8554698
LOSS: 35012.1489823
LOSS: 34987.7529984
LOSS: 34965.6029149
LOSS: 34945.1154625
LOSS: 34925.4606044
LOSS: 34906.6308513
LOSS: 34888.3213515
LOSS: 34870.3691298
LOSS: 34852.4730644
LOSS: 34834.4306608
LOSS: 34816.1197776
LOSS: 34797.4300523
LOSS: 34778.3375476
LOSS: 34758.8532834
LOSS: 34739.0632689
LOSS: 34719.1263735
LOSS: 34699.2043933
LOSS: 34679.4349165
LOSS: 34660.1367422
LOSS: 34641.5533585
LOSS: 34623.7517772
LOSS: 34606.8343138
LOSS: 34590.8016614
LOSS: 34575.6565838
LOSS: 34561.3969959
LOSS: 34548.0189755
LOSS: 34535.4956908
LOSS: 34523.7883391
LOSS: 34512.860846
LOSS: 34502.6612235
LOSS: 34493.1466188
LOSS: 34484.2525946
LOSS: 34475.9184017
LOSS: 34468.1014859
LOSS: 34460.7668717
LOSS: 34453.8825572
LOSS: 34447.4090776
LOSS: 34441.311189
LOSS: 34435.5574457
LOSS: 34430.1347659
LOSS: 34425.0117852
LOSS:

Lamda:  2.858613
LOSS: 34333.4771532
Lamda:  2.858613 RMSE:  1.6015804759 Val RMSE: 1.94666082216
Lamda:  2.858611
LOSS: 34333.4702827
Lamda:  2.858611 RMSE:  1.60157768913 Val RMSE: 1.94665881178
Lamda:  2.858609
LOSS: 34333.4634506
Lamda:  2.858609 RMSE:  1.60157496856 Val RMSE: 1.94665685106
Lamda:  2.858607
LOSS: 34333.4566551
Lamda:  2.858607 RMSE:  1.60157231253 Val RMSE: 1.94665493931
Lamda:  2.858605
LOSS: 34333.4498944
Lamda:  2.858605 RMSE:  1.60156971947 Val RMSE: 1.94665307541
Lamda:  2.858603
LOSS: 34333.443167
Lamda:  2.858603 RMSE:  1.60156718787 Val RMSE: 1.94665125814
Lamda:  2.858601
LOSS: 34333.436471
Lamda:  2.858601 RMSE:  1.60156471627 Val RMSE: 1.94664948625
Lamda:  2.858599
LOSS: 34333.4298049
Lamda:  2.858599 RMSE:  1.60156230081 Val RMSE: 1.94664772939
Lamda:  2.858597
LOSS: 34333.4231675
Lamda:  2.858597 RMSE:  1.60155993355 Val RMSE: 1.94664599263
Lamda:  2.858595
LOSS: 34333.4165572
Lamda:  2.858595 RMSE:  1.60155761548 Val RMSE: 1.94664430209
Lamda:  2.858

Lamda:  2.858447
LOSS: 34332.9567085
Lamda:  2.858447 RMSE:  1.60147664856 Val RMSE: 1.94658901594
Lamda:  2.858445
LOSS: 34332.9506186
Lamda:  2.858445 RMSE:  1.60147620776 Val RMSE: 1.94658876575
Lamda:  2.858443
LOSS: 34332.9445292
Lamda:  2.858443 RMSE:  1.60147577518 Val RMSE: 1.94658852178
Lamda:  2.858441
LOSS: 34332.9384405
Lamda:  2.858441 RMSE:  1.60147535064 Val RMSE: 1.94658828388
Lamda:  2.858439
LOSS: 34332.9323523
Lamda:  2.858439 RMSE:  1.60147493392 Val RMSE: 1.94658805188
Lamda:  2.858437
LOSS: 34332.9262646
Lamda:  2.858437 RMSE:  1.60147452483 Val RMSE: 1.94658782565
Lamda:  2.858435
LOSS: 34332.9201774
Lamda:  2.858435 RMSE:  1.60147412318 Val RMSE: 1.94658760504
Lamda:  2.858433
LOSS: 34332.9140907
Lamda:  2.858433 RMSE:  1.60147372878 Val RMSE: 1.9465873899
Lamda:  2.858431
LOSS: 34332.9080044
Lamda:  2.858431 RMSE:  1.60147334145 Val RMSE: 1.94658718011
Lamda:  2.858429
LOSS: 34332.9019185
Lamda:  2.858429 RMSE:  1.60147296102 Val RMSE: 1.94658697552
Lamda:  2.8

LOSS: 34332.4520131
Lamda:  2.858281 RMSE:  1.60145583248 Val RMSE: 1.94658015741
Lamda:  2.858279
LOSS: 34332.4459351
Lamda:  2.858279 RMSE:  1.60145568074 Val RMSE: 1.94658012544
Lamda:  2.858277
LOSS: 34332.4398572
Lamda:  2.858277 RMSE:  1.60145553001 Val RMSE: 1.94658009423
Lamda:  2.858275
LOSS: 34332.4337792
Lamda:  2.858275 RMSE:  1.60145538026 Val RMSE: 1.94658006376
Lamda:  2.858273
LOSS: 34332.4277013
Lamda:  2.858273 RMSE:  1.60145523147 Val RMSE: 1.94658003402
Lamda:  2.858271
LOSS: 34332.4216234
Lamda:  2.858271 RMSE:  1.60145508361 Val RMSE: 1.94658000498
Lamda:  2.858269
LOSS: 34332.4155455
Lamda:  2.858269 RMSE:  1.60145493667 Val RMSE: 1.94657997662
Lamda:  2.858267
LOSS: 34332.4094676
Lamda:  2.858267 RMSE:  1.60145479061 Val RMSE: 1.94657994894
Lamda:  2.858265
LOSS: 34332.4033897
Lamda:  2.858265 RMSE:  1.60145464542 Val RMSE: 1.94657992191
Lamda:  2.858263
LOSS: 34332.3973118
Lamda:  2.858263 RMSE:  1.60145450107 Val RMSE: 1.94657989551
Lamda:  2.858261
LOSS: 3433

Lamda:  2.858113
LOSS: 34331.9414708
Lamda:  2.858113 RMSE:  1.60144505417 Val RMSE: 1.94657895642
Lamda:  2.858111
LOSS: 34331.9353929
Lamda:  2.858111 RMSE:  1.60144493798 Val RMSE: 1.94657895127
Lamda:  2.858109
LOSS: 34331.929315
Lamda:  2.858109 RMSE:  1.60144482191 Val RMSE: 1.94657894621
Lamda:  2.858107
LOSS: 34331.923237
Lamda:  2.858107 RMSE:  1.60144470596 Val RMSE: 1.94657894124
Lamda:  2.858105
LOSS: 34331.9171591
Lamda:  2.858105 RMSE:  1.60144459013 Val RMSE: 1.94657893635
Lamda:  2.858103
LOSS: 34331.9110811
Lamda:  2.858103 RMSE:  1.6014444744 Val RMSE: 1.94657893155
Lamda:  2.858101
LOSS: 34331.9050032
Lamda:  2.858101 RMSE:  1.60144435879 Val RMSE: 1.94657892684
Lamda:  2.858099
LOSS: 34331.8989253
Lamda:  2.858099 RMSE:  1.60144424328 Val RMSE: 1.9465789222
Lamda:  2.858097
LOSS: 34331.8928473
Lamda:  2.858097 RMSE:  1.60144412788 Val RMSE: 1.94657891764
Lamda:  2.858095
LOSS: 34331.8867693
Lamda:  2.858095 RMSE:  1.60144401257 Val RMSE: 1.94657891316
Lamda:  2.8580

...

LOSS: 34319.78671
Lamda:  2.854115 RMSE:  1.6012223864 Val RMSE: 1.94657603957
Lamda:  2.854113
LOSS: 34319.7806271
Lamda:  2.854113 RMSE:  1.60122227517 Val RMSE: 1.94657603834
Lamda:  2.854111
LOSS: 34319.7745441
Lamda:  2.854111 RMSE:  1.60122216394 Val RMSE: 1.94657603711
Lamda:  2.854109
LOSS: 34319.7684612
Lamda:  2.854109 RMSE:  1.60122205271 Val RMSE: 1.94657603588
Lamda:  2.854107
LOSS: 34319.7623783
Lamda:  2.854107 RMSE:  1.60122194148 Val RMSE: 1.94657603465
Lamda:  2.854105
LOSS: 34319.7562954
Lamda:  2.854105 RMSE:  1.60122183025 Val RMSE: 1.94657603342
Lamda:  2.854103
LOSS: 34319.7502124
Lamda:  2.854103 RMSE:  1.60122171902 Val RMSE: 1.94657603219
Lamda:  2.854101
LOSS: 34319.7441295
Lamda:  2.854101 RMSE:  1.60122160779 Val RMSE: 1.94657603096
Lamda:  2.854099
LOSS: 34319.7380466
Lamda:  2.854099 RMSE:  1.60122149656 Val RMSE: 1.94657602973
Lamda:  2.854097
LOSS: 34319.7319636
Lamda:  2.854097 RMSE:  1.60122138534 Val RMSE: 1.9465760285
Lamda:  2.854095
LOSS: 34319.72

LOSS: 34319.2818194
Lamda:  2.853949 RMSE:  1.60121315461 Val RMSE: 1.94657593773
Lamda:  2.853947
LOSS: 34319.2757363
Lamda:  2.853947 RMSE:  1.60121304339 Val RMSE: 1.94657593651
Lamda:  2.853945
LOSS: 34319.2696532
Lamda:  2.853945 RMSE:  1.60121293217 Val RMSE: 1.94657593528
Lamda:  2.853943
LOSS: 34319.26357
Lamda:  2.853943 RMSE:  1.60121282094 Val RMSE: 1.94657593406
Lamda:  2.853941
LOSS: 34319.2574869
Lamda:  2.853941 RMSE:  1.60121270972 Val RMSE: 1.94657593284
Lamda:  2.853939
LOSS: 34319.2514038
Lamda:  2.853939 RMSE:  1.6012125985 Val RMSE: 1.94657593161
Lamda:  2.853937
LOSS: 34319.2453206
Lamda:  2.853937 RMSE:  1.60121248727 Val RMSE: 1.94657593039
Lamda:  2.853935
LOSS: 34319.2392375
Lamda:  2.853935 RMSE:  1.60121237605 Val RMSE: 1.94657592917
Lamda:  2.853933
LOSS: 34319.2331544
Lamda:  2.853933 RMSE:  1.60121226483 Val RMSE: 1.94657592794
Lamda:  2.853931
LOSS: 34319.2270712
Lamda:  2.853931 RMSE:  1.6012121536 Val RMSE: 1.94657592672
Lamda:  2.853929
LOSS: 34319.22

LOSS: 34318.7769117
Lamda:  2.853783 RMSE:  1.60120392331 Val RMSE: 1.94657583649
Lamda:  2.853781
LOSS: 34318.7708283
Lamda:  2.853781 RMSE:  1.60120381209 Val RMSE: 1.94657583527
Lamda:  2.853779
LOSS: 34318.764745
Lamda:  2.853779 RMSE:  1.60120370088 Val RMSE: 1.94657583406
Lamda:  2.853777
LOSS: 34318.7586617
Lamda:  2.853777 RMSE:  1.60120358966 Val RMSE: 1.94657583284
Lamda:  2.853775
LOSS: 34318.7525783
Lamda:  2.853775 RMSE:  1.60120347844 Val RMSE: 1.94657583163
Lamda:  2.853773
LOSS: 34318.746495
Lamda:  2.853773 RMSE:  1.60120336722 Val RMSE: 1.94657583041
Lamda:  2.853771
LOSS: 34318.7404116
Lamda:  2.853771 RMSE:  1.60120325601 Val RMSE: 1.9465758292
Lamda:  2.853769
LOSS: 34318.7343283
Lamda:  2.853769 RMSE:  1.60120314479 Val RMSE: 1.94657582798
Lamda:  2.853767
LOSS: 34318.7282449
Lamda:  2.853767 RMSE:  1.60120303357 Val RMSE: 1.94657582676
Lamda:  2.853765
LOSS: 34318.7221616
Lamda:  2.853765 RMSE:  1.60120292236 Val RMSE: 1.94657582555
Lamda:  2.853763
LOSS: 34318.7

LOSS: 34318.2719867
Lamda:  2.853617 RMSE:  1.60119469249 Val RMSE: 1.94657573585
Lamda:  2.853615
LOSS: 34318.2659032
Lamda:  2.853615 RMSE:  1.60119458128 Val RMSE: 1.94657573464
Lamda:  2.853613
LOSS: 34318.2598196
Lamda:  2.853613 RMSE:  1.60119447007 Val RMSE: 1.94657573344
Lamda:  2.853611
LOSS: 34318.2537361
Lamda:  2.853611 RMSE:  1.60119435886 Val RMSE: 1.94657573223
Lamda:  2.853609
LOSS: 34318.2476525
Lamda:  2.853609 RMSE:  1.60119424765 Val RMSE: 1.94657573102
Lamda:  2.853607
LOSS: 34318.241569
Lamda:  2.853607 RMSE:  1.60119413644 Val RMSE: 1.94657572981
Lamda:  2.853605
LOSS: 34318.2354854
Lamda:  2.853605 RMSE:  1.60119402523 Val RMSE: 1.9465757286
Lamda:  2.853603
LOSS: 34318.2294019
Lamda:  2.853603 RMSE:  1.60119391401 Val RMSE: 1.94657572739
Lamda:  2.853601
LOSS: 34318.2233183
Lamda:  2.853601 RMSE:  1.6011938028 Val RMSE: 1.94657572619
Lamda:  2.853599
LOSS: 34318.2172348
Lamda:  2.853599 RMSE:  1.60119369159 Val RMSE: 1.94657572498
Lamda:  2.853597
LOSS: 34318.2

KeyboardInterrupt: 

In [ ]:
#model.Lamda = 2.8585
#model.chooseCorrectLamda()
#model.saveModel('modelT')
#model.Lamda = 2.2
#model.chooseCorrectLamda()
#1.94657660345
#1.94657669378
#1.9465770994
#1.94657820105 : 2.857295
#

In [ ]:
#model.saveModel('model2826_1.94663989568')

In [ ]:
#model.Lamda = 2.83
#model.chooseCorrectLamda()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.trainlamda, model.trainrmse)
plt.plot(model.vallamda, model.valrmse)
plt.ylabel('RMSE')
plt.xlabel('Lamda')
ax = plt.gca()
ax.invert_xaxis()
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
#plt.savefig('RMSEvsLamda.png')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.NonZero)
plt.ylabel('Non Zero')
plt.xlabel('Iterations')
#plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
print model.NonZero
print np.count_nonzero(model.W)
#plt.savefig('NonZeroElements.png')

In [ ]:
model.saveModel('savedModel244')

In [ ]:
#print model.lamda
print rmse(model.predict(valX), valY)
print rmse(X.transpose() * model.W, valY)
print model.predict(valX)
print valY

In [ ]:
testXDF = pd.read_csv('Data/testData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
testX = csr_matrix((valXDF['value'], (valXDF['featureID'], valXDF['instanceID'])))
testPredicted = model.predict(testX)
print testPredicted
np.savetxt("out3.csv", testPredicted, delimiter=",")
#testPredicted.to_csv('out.csv')

In [ ]:
# Mixture
bigData = (X + valX) / 2


In [ ]:
loadedModel = loadModel('./finalData/savedModel')
plt.plot(loadedModel.NonZero)
plt.ylabel('Non Zero')
plt.xlabel('Iterations')
plt.show()

In [71]:
print loadedModel.W
WLoaded = list(loadedModel.W)
sorted(range(len(WLoaded)), key=lambda i: WLoaded[i])[-10:]
print WLoaded[2468]
print max(WLoaded)

[ 0.          3.70430265  5.27646819 ..., -0.17621468  0.91925534
  1.82692104]
8.76831112942
11.2180665852


In [52]:
#print loadedModel.trainlamda[1:]
#print loadedModel.trainrmse.shape
#plt.plot(loadedModel.trainlamda[1:], loadedModel.trainrmse)
##plt.plot(loadedModel.vallamda[1:], loadedModel.valrmse)
##plt.ylabel('RMSE')
#plt.xlabel('Lamda')
#ax = plt.gca()
#ax.invert_xaxis()
#plt.legend(['Train', 'Validation'], loc='upper right')
#plt.show()

In [31]:
print valModel.Lamda

2.858499


In [21]:
valModel = Lasso(valX, valY, W, B, model.Lamda)


In [22]:
#valModel.Lamda = 2.4466

valModel.fit()

Lamda:  2.853579
LOSS: 45062.4759015
LOSS: 45953.9486257


In [23]:
#print model.lamda
print rmse(model.predict(valX), valY)
print rmse(X.transpose() * model.W, valY)
print model.predict(valX)
print valY
print rmse(valY, (valModel.X.transpose() * valModel.W) + np.full(valModel.X.transpose().shape[0], valModel.B[0]))
print rmse(Y, (X.transpose() * valModel.W) + np.full(X.transpose().shape[0], valModel.B[0]))


1.60618898831
83.1680666329
[ 92.47196241  91.38830316  86.23788172 ...,  91.89792193  87.39086242
  88.87540914]
[92 90 86 ..., 92 89 88]
1.60611322806
1.92234375686


In [24]:
#print model.lamda
print rmse(model.predict(valX), valY)
print rmse(X.transpose() * model.W, valY)
print model.predict(valX)
print valY
print rmse(valY, (valModel.X.transpose() * valModel.W) + np.full(valModel.X.transpose().shape[0], valModel.B[0]))
print rmse(Y, (X.transpose() * valModel.W) + np.full(X.transpose().shape[0], valModel.B[0]))


1.60618898831
83.1680666329
[ 92.47196241  91.38830316  86.23788172 ...,  91.89792193  87.39086242
  88.87540914]
[92 90 86 ..., 92 89 88]
1.60611322806
1.92234375686


In [116]:
#print model.lamda
print rmse(model.predict(valX), valY)
print rmse(X.transpose() * model.W, valY)
print model.predict(valX)
print valY
print rmse(valY, (valModel.X.transpose() * valModel.W) + np.full(valModel.X.transpose().shape[0], valModel.B[0]))
print rmse(Y, (X.transpose() * valModel.W) + np.full(X.transpose().shape[0], valModel.B[0]))


2.3258676229
81.1925424054
[ 93.95461517  92.93189006  87.62378878 ...,  92.54065589  89.5128428
  89.97589808]
[92 90 86 ..., 92 89 88]
1.46293862824
2.0969385907


In [25]:
testXDF = pd.read_csv('Data/testData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
testXDF['instanceID'] -= 1
testXDF['featureID'] -= 1
testX = csr_matrix((testXDF['value'], (testXDF['featureID'], testXDF['instanceID'])))
print testX.shape[0]
testPredicted = (testX.transpose() * valModel.W) + np.full(testX.transpose().shape[0], valModel.B[0])
#np.full(X.transpose().shape[0], self.B)
#testPredicted = valModel.predict(testX)
print testPredicted
np.savetxt("out6.csv", testPredicted, delimiter=",", )

3000
[ 90.74299784  82.71997622  88.61618647 ...,  88.27129239  87.97372036
  87.20146213]


In [35]:
t1 = pd.read_csv('./out1.csv', sep=',')
t2 = pd.read_csv('./out5.csv', sep=',')
#print t1

In [36]:
rmse(t1['Points'], t2['Points'])

0.3347943645426332